#프로젝트

### Step 1. 형태소 분석기 변경해 보기
한국어 자연어 처리에 성능에 영향을 주는 또 다른 요소는 사용하는 형태소 분석기의 종류입니다. 앞서 진행한 프로젝트에서는 Mecab을 사용했었습니다. 하지만 이번에 설치한 konlpy 패키지 안에는 Mecab 이외에도 Hannanum, Kkma, Komoran, Okt 라는 형태소분석기가 존재합니다. 이들 형태소 분석기 중 어떤 것을 선택하는 것이 좋을지, 형태소 분석기간의 장단점은 무엇인지, 처리 속도와 성능에는 어떤 변화가 있는지 비교해 보세요.


### Step 2. 불용어 추가해 보기
머신 러닝을 하다보면 전처리는 어쩌면 끝이 없는 작업일 수도 있습니다. 불용어 추가도 마찬가지입니다. 데이터를 출력해 보고, 직접 뉴스를 열어보면서 불용어가 총 50개가 될 때까지 추가해 보세요. 그리고 불용어를 추가하기 전, 후의 모델 성능을 비교해 보세요.

지나치게 빈도가 낮은 단어, 지나치게 빈도가 높은 단어 둘 중 어떤 단어가 불용어로 선택할 경우에 모델 성능 향상에 효과가 더 좋을지도 고민해 봅시다.

### Step 3. 다른 날짜 데이터 추가해 보기
동일한 날짜에는 그 날 특정 사건이 일어나서 비슷한 뉴스들이 올라왔을 가능성이 많습니다. 사실 좀 더 정확한 모델을 만들기 위해서는 다른 날짜의 뉴스들도 훈련 데이터로 사용하는 것이 좋을 겁니다.

크롤링을 위한 코드는 이미 주어졌습니다. 이제는 2020년 5월 6일 외에도 다른 일자의 데이터를 좀 더 추가해 보겠습니다. 이미 수집한 2020년 5월 6일 데이터와 추가로 수집된 데이터를 적절히 합치려면 어떻게 해야할까요? 또 어떻게 해야 적절히 데이터를 섞을 수 있을까요? 한번 고민해서 데이터를 수집해 보세요.

[안녕](#동일


#### library import

In [61]:
# import library


import requests
import pandas as pd
from bs4 import BeautifulSoup
from newspaper import Article
import os

# 형태소 분석기
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
from konlpy.tag import Mecab
#init_jvm() 

#머신러닝...
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics



#### 데이터 추출 

In [62]:
##### 데이터 추출 함수 정의

In [63]:


# 페이지 수, 카테고리, 날짜를 입력값으로 받습니다.
def make_urllist(page_num, code, date): 
  urllist= []
  for i in range(1, page_num + 1):
    url = 'https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1='+str(code)+'&date='+str(date)+'&page='+str(i)   
    news = requests.get(url)

    # BeautifulSoup의 인스턴스 생성합니다. 파서는 html.parser를 사용합니다.
    soup = BeautifulSoup(news.content, 'html.parser')

    # CASE 1
    news_list = soup.select('.newsflash_body .type06_headline li dl')
    #print(news_list)
    # CASE 2
    news_list.extend(soup.select('.newsflash_body .type06 li dl'))
        
    # 각 뉴스로부터 a 태그인 <a href ='주소'> 에서 '주소'만을 가져옵니다.
    for line in news_list:
        urllist.append(line.a.get('href'))
  return urllist

#- 데이터프레임을 생성하는 함수입니다.
def make_data(urllist, code):
  text_list = []
  for url in urllist:
    article = Article(url, language='ko')
    article.download()
    article.parse()
    text_list.append(article.text)

  #- 데이터프레임의 'news' 키 아래 파싱한 텍스트를 밸류로 붙여줍니다.
  df = pd.DataFrame({'news': text_list})

  #- 데이터프레임의 'code' 키 아래 한글 카테고리명을 붙여줍니다.
  df['code'] = idx2word[str(code)]
  return df

def make_total_data(page_num, code_list, date):
    df = None

    for code in code_list:
        url_list = make_urllist(page_num, code, date) #BeautifulSoup select
        df_temp = make_data(url_list, code) #newspaer title conntent export
        print(str(code)+'번 코드에 대한 데이터를 만들었습니다.')

        if df is not None:
          df = pd.concat([df, df_temp]) # date 연결 이어 붙임 
        else:
          df = df_temp

    return df

#### 추출한 데이터 csv 파일로 저장

- 이미 제공된 파일로 대체 주석처리..


In [64]:
# 아래 주석처리된 코드의 주석을 해제하고 실행을 하면 대량 크롤링이 진행됩니다. 
# 위에서 수행했던 크롤링의 100배 분량이 수행될 것입니다. 한꺼번에 너무 많은 크롤링 요청이 서버에 전달되지 않도록 주의해 주세요. 
# 기사 일자를 바꿔보면서 데이터를 모으면 더욱 다양한 데이터를 얻을 수 있게 됩니다. 

#df = make_total_data(100, code_list, 20200506)

# 

# # 데이터프레임 파일을 csv 파일로 저장합니다.
# # 저장경로는 이번 프로젝트를 위해 만든 폴더로 지정해 주세요.

# csv_path = os.getenv("./news_data.csv")

# df.to_csv(csv_path, index=False)

# if os.path.exists(csv_path):
#   print('{} File Saved!'.format(csv_path))

#### 데이터 정제

In [194]:

### step3 다른 데이터
csv_path = "./news_data.csv" #파일 path ./news_data2.csv

df = pd.read_table(csv_path, sep=',')  #파일 읽어드리기
df.head()

,news,code
0,파주시청. 사진제공=파주시 파주시청. 사진제공=파주시\n\n[파주=파이낸셜뉴스 강근...,사회
1,동영상 뉴스\n\n이천 물류창고 화재 발화지점으로 지목된 지하 2층에서 산소절단기의...,사회
2,황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검. 사진...,사회
3,귀갓길 여성을 쫓아가 성범죄를 시도한 20대 남성이 구속됐습니다.서울 강남경찰서는 ...,사회
4,(서울=연합뉴스) 대한약사회가 6일부터 코로나바이러스 감염증 대응 체계를 '사회적 ...,사회


#### 한글 외 특수만자, 개행문자 등 제거

In [195]:
# 정규 표현식을 이용해서 한글 외의 문자는 전부 제거합니다.
# 개행 문자....
df['news'] = df['news'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
df['news']

0       파주시청 사진제공파주시 파주시청 사진제공파주시파주파이낸셜뉴스 강근주 기자 파주시는 ...
1       동영상 뉴스이천 물류창고 화재 발화지점으로 지목된 지하 층에서 산소절단기의 산소 공...
2       황범순 의정부시 부시장 을지대학교 의정부캠퍼스 및 부속병원 공사현장 안전점검 사진제...
3       귀갓길 여성을 쫓아가 성범죄를 시도한 대 남성이 구속됐습니다서울 강남경찰서는 강간상...
4       서울연합뉴스 대한약사회가 일부터 코로나바이러스 감염증 대응 체계를 사회적 거리두기에...
                              ...                        
5119    신종 코로나바이러스 감염증코로나 사태 이후 가정의 달 월에도 언택트비대면 신풍속도가...
5120    는 소비자로부터 월 이용료 만만원을 받고 초고속 인터넷을 제공한다 그런 브로드밴드가...
5121    머리를 긁고 있는 오랑우탄 몸을 긁는 행동을 따라 하는 것은 부정적 감정과 관련이 ...
5122    가 오는 일 정식 출시하는 스마트폰 벨벳이 사실상 공짜폰이 될 전망이다 단말기 가격...
5123    이미지제공게티이미지뱅크 이미지제공게티이미지뱅크  전자신문  전자신문인터넷 무단전재 ...
Name: news, Length: 5124, dtype: object

#### 데이터 체크

In [196]:
# null 값 개수 조회
print(df.isnull().sum())

news    0
code    0
dtype: int64


In [197]:
print('뉴스 기사의 개수: ',len(df))
# 중복된 샘플들을 제거합니다.
df.drop_duplicates(subset=['news'], inplace=True)

print('뉴스 기사의 개수: ',len(df))

뉴스 기사의 개수:  5124
뉴스 기사의 개수:  3994


#### 데이터 탐색

In [198]:
print(df.groupby('code').size().reset_index(name = 'count'))

    code  count
0  IT/과학    903
1     사회   1668
2  생활/문화   1423


#### 토큰화 (mecab) ,  형태소 분석 토큰화 및 불용어 제거

In [286]:



#형태소 분석기 정하기

#tokenizer = Mecab()
#tokenizer = Hannanum()
#tokenizer = Komoran()
tokenizer = Okt()

#Hannanum, Kkma, Komoran, Okt

# 불용어
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']


#불용어 추가
stopwords.extend(['우리', '으로','나오','가지','씨','시키','생각하','모르','소리','사실', '통하','좀','원','이렇', '더','더욱더', '그런','자주','종종','가끔','해야','있기','제보','검색','배포','재배포','금지', '네이버'])

In [287]:
# 토큰화 함수 정의
# 토큰화 및 토큰화 과정에서 불용어를 제거하는 함수입니다.
def preprocessing(data):
  text_data = []

  for sentence in data:
    temp_data = []
    #- 토큰화
    temp_data = tokenizer.morphs(sentence) 
    #- 불용어 제거
    temp_data = [word for word in temp_data if not word in stopwords] 
    text_data.append(temp_data)

  text_data = list(map(' '.join, text_data))

  return text_data

In [288]:
# 토큰화 실행
text_data = preprocessing(df['news'])
text_data[0]

'파주 시청 사진 제공 파주시 파주 시청 사진 제공 파주시 파주 파이낸셜뉴스 강 근 주 파주시 일 관내 취약 계층 만 가구 대해 정부 긴급 재난 지 원금 입금 완료 했다 파주시민 받는 긴급 재난 지원 금은 인 이상 가구 기준 만원 받게 되며 인 가구 만원 인 가구 만원 인 가구 만원 이다 정부 발표 긴급 재난 지 원금 파주시민 지급 금액 다른 이유 국비 지 방비 부담 비율 이다 파주시 이미 모든 시민 경기도 파주시 재난 기본소득 인 당 각 만원 지급 하고 있어 시민 국비 지 원금 만 지급 하며 인 가구 기준 총 지원 금액 파주시 재난 기본소득 만원 경기도 재난 기본소득 만원 정부 긴급 재난 지 원금 만원 총 만원 받게 된다 취약 계층 아닌 시민 오는 월일 부터 소지 하고 있는 신용 체크카드 사 홈페이지 에서 긴급 재난 지 원금 지원 신청 있다 세대주 가족 지 원금 일괄 신청 한다 한편 파주시 일 김정기 부시장 단장 하는 긴급 재난 지 원금 추진 태스크포스 를 구성 해 긴급 재난 지 원금 원활하게 지급 될 있도록 지원 저작권 자 파이낸셜뉴스 전 재재'

#### 훈련 데이터 테스트 데이터 정의

In [289]:
#- 훈련 데이터와 테스트 데이터를 분리합니다.
X_train, X_test, y_train, y_test = train_test_split(text_data, df['code'], random_state = 12, test_size=0.2)

In [290]:
print('훈련용 뉴스 기사의 개수 :', len(X_train))
print('테스트용 뉴스 기사의 개수 : ', len(X_test))
print('훈련용 레이블의 개수 : ', len(y_train))
print('테스트용 레이블의 개수 : ', len(y_test))

훈련용 뉴스 기사의 개수 : 3195
테스트용 뉴스 기사의 개수 :  799
훈련용 레이블의 개수 :  3195
테스트용 레이블의 개수 :  799


In [291]:
#### 텍스트를 입력하면 자동으로 TF-IDF 벡터로 바꾸는 전처리 함수

In [292]:

def tfidf_vectorizer(data):
  data_counts = count_vect.transform(data)
  data_tfidf = tfidf_transformer.transform(data_counts)
  return data_tfidf

#### 단어수 카운트 및 백터라이져 TF-IDF , 나이브 베이즈 분류기 수행

In [293]:
#- 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저입니다.
# 단어 데이터를 학습하고 문서 데이터를 document-form matrix로 변환하는 두 가지 작업을 해 줍니다.
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

#print(X_train_counts)

#- 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#print(X_train_tfidf)

#- 나이브 베이즈 분류기를 수행합니다.
#- X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)


#### 모델을 예측하게하고, 실제값과 비교하여 점수를 측정

In [294]:
#

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       IT/과학       0.92      0.81      0.86       168
          사회       0.76      0.93      0.84       336
       생활/문화       0.87      0.72      0.79       295

    accuracy                           0.82       799
   macro avg       0.85      0.82      0.83       799
weighted avg       0.84      0.82      0.82       799



![nn](imgs/d2-okt.png)




##

- 한번에 하면 너무 오래 기다려야 되서 하나씩하면서
- imgs 폴더에 결과 저장했습니다. 
- 데이터명 - 분석기명.png
- d1 :data1
- d2 : 다른 날짜 data


